# Donkey Car IMU Model

Build and run a Keras model copied from Donkey Car.

* The IMU model has been modified to accept larger telemetry inputs
* RNN-LSTM has been modified to accept IMU/telemetry

In [12]:
## Imports
import numpy as np
import pandas as pd
import tensorflow as tf
import time
import pickle
import sklearn.metrics as metrics
from os.path import exists

from modeling_methods import run_model, plot_metrics, save_model, create_donkey_vimu
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from tensorflow.keras.metrics import MAE, MSE, RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

## Directories/Variables

In [2]:
## Fill in your own here
dataset_directory = '../data/01_28_2022/11_34_57'

In [3]:
## Directories
model_directory = f'../models'

## File paths
cam_input_dataset_file = f'{dataset_directory}/X_img.npy'
telem_input_dataset_file = f'{dataset_directory}/X_telem.pkl'
target_dataset_file = f'{dataset_directory}/y.npy'

## Parameters
scaler_type = 'robust' # minmax/robust/standard 
batch_sizes = [1, 2, 4, 8, 16, 32, 64, 128, 256 , 512, 1024, 2048]
# batch_sizes = [16, 32, 64, 128, 256, 512, ] # 1024, 2048]
early_stop_patience = 5 # None for no stop
epochs = 250
dual_outputs = False
create_model = create_donkey_vimu

In [4]:
scalers = {
    'minmax': MinMaxScaler,
    'robust': RobustScaler,
    'standard': StandardScaler,
}

## Data

### Load Datasets

In [5]:
## Load the datasets
X_cam = np.load(cam_input_dataset_file).astype('uint8')

## Load telemetry as df to grab telemetry column names
telem_df = pd.read_pickle(telem_input_dataset_file)
telemetry_columns = telem_df.columns
## Convert to numpy, delete df
X_telem = telem_df.to_numpy().astype('float32')
del telem_df

## Load targets
y = np.load(target_dataset_file, mmap_mode='r')

## Check Shape
X_cam.shape

(214894, 120, 160, 1)

### Train-Test Split

In [6]:
if dual_outputs:
    # steering = y[:, 0], throttle = y[:, 1]
    datasets = train_test_split(X_cam, X_telem, y[:, 0], y[:, 1], test_size=0.2, random_state=0)
else:
    datasets = train_test_split(X_cam, X_telem, y, test_size=0.2, random_state=0)

del X_telem
del X_cam
    
X_cam_train = datasets[0]
X_cam_test = datasets[1]
X_telem_train = datasets[2]
X_telem_test = datasets[3]
y_train = datasets[4]
y_test = datasets[5]
    
if dual_outputs:
    y_st_train = datasets[4]
    y_st_test = datasets[5]
    y_th_train = datasets[6]
    y_th_test = datasets[7]

### Scale IMU Data

In [7]:
scaler_file = f'{scaler_type}_scaler_{time.strftime("%m_%d_%H_%M")}.pkl'
scaler_path = f'../scalers/{scaler_file}'

sc = scalers[scaler_type]()
    
## Fit to then and transform training data
X_telem_train_sc = sc.fit_transform(X_telem_train)
## Transform testing data
X_telem_test_sc = sc.transform(X_telem_test)

### Save the Scaler for Predictions

In [8]:
## Save as pickle
pickle.dump(sc, open(scaler_path, 'wb'))

## Print path
scaler_file

'robust_scaler_01_28_11_45.pkl'

### Get Input Shape(s)

In [9]:
## Create variables
img_input_shape = X_cam_train[0].shape
tel_input_shape = X_telem_train_sc[0].shape

## Check telemetry input shape
img_input_shape

(120, 160, 1)

In [10]:
X_cam_train.shape

(171915, 120, 160, 1)

In [11]:
X_telem_train_sc.shape[1]

4

In [13]:
X_cam_train_tensor = tf.constant(X_cam_train.astype('uint8'))
X_telem_train_tensor = tf.constant(X_telem_train_sc.astype('uint8'))
X_cam_test_tensor = tf.constant(X_cam_test.astype('float32'))
X_telem_test_tensor = tf.constant(X_telem_test_sc.astype('float32'))
y_train_tensor = tf.constant(y_train.astype('float32'))
y_test_tensor = tf.constant(y_test.astype('float32'))

2022-01-28 11:46:00.218759: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-28 11:46:00.235220: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-28 11:46:00.322705: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-28 11:46:00.323023: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1060 6GB computeCapability: 6.1
coreClock: 1.797GHz coreCount: 10 deviceMemorySize: 5.93GiB deviceMemoryBandwidth: 178.99GiB/s
2022-01-28 11:46:00.323050: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-01-28 11:46:00.484085: I tensorflow/stream_executor/platfor

In [ ]:
# X_train_ds = tf.data.Dataset.from_tensor_slices((X_cam_train_tensor, X_telem_train_tensor))
# X_test_ds = tf.data.Dataset.from_tensor_slices((X_cam_test_tensor, X_telem_test_tensor))

In [ ]:
# tf.RaggedTensor.from_tensor(X_cam_train_tensor)

In [ ]:
# X_cam_train_tensor[None].shape

In [ ]:
# train_data = tf.data.Dataset.from_tensor_slices(({"img_in": X_cam_train, "tel_in": X_telem_train_sc}, y_train))
# test_data = tf.data.Dataset.from_tensor_slices(({"img_in": X_cam_test, "tel_in": X_telem_test_sc}, y_test))

In [ ]:
# train_data = tf.data.Dataset.from_tensor_slices(({"img_in": X_cam_train, "tel_in": X_telem_train_sc}, y_train))
# test_data = tf.data.Dataset.from_tensor_slices(({"img_in": X_cam_test, "tel_in": X_telem_test_sc}, y_test))

In [ ]:
# train_dataset = tf.data.Dataset.from_tensor_slices((X_train_ds, y_train_tensor))
# test_dataset = tf.data.Dataset.from_tensor_slices((X_test_ds, y_test_tensor))

In [ ]:
# test_dataset = tf.data.Dataset.from_tensor_slices((tf.ragged.constant([X_test, y_test]), y_test))

## Model Loop

In [ ]:
X_train = (X_cam_train_tensor, X_telem_train_tensor)
X_test = (X_cam_test_tensor, X_telem_test_tensor)
y_train = y_train_tensor
y_test = y_test_tensor

# X_train = (X_cam_train.astype('float32'), X_telem_train_sc.astype('float32'))
# X_test = (X_cam_test.astype('float32'), X_telem_test_sc.astype('float32'))
# y_train = y_train.astype('float32')
# y_test = y_test.astype('float32')


# if dual_outputs:
#     y_train = (y_st_train, y_th_train)
#     y_test = (y_st_test, y_th_test)
# else:
#     y_train = y_train
#     y_test = y_test
    
# train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
# test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))


## Run models for each batch size
print('---')
for batch_size in batch_sizes:
    print(f'Batch size {batch_size} start: {time.strftime("%H:%M:%S")}')
    model = create_model(img_input_shape, tel_input_shape, dual_outputs)
    model.compile(loss='mse', 
                  optimizer=Adam(learning_rate=0.0001), 
                  metrics=['mae', RootMeanSquaredError()])
    model, results = run_model(model, X_train, y_train, X_test, y_test, 
                               batch_size, epochs,
                              early_stop_patience=early_stop_patience)
    # model, results = run_model(model, train_data, test_data, 
    #                        batch_size, epochs,
    #                       early_stop_patience=early_stop_patience)

    model_file = save_model(model_directory, model, results, batch_size, 
                            dual_outputs, scaler_file, telemetry_columns,
                            dataset_directory)
    history = {k: v for k, v in results.history.items()}
    plot_metrics(history, batch_size, dual_outputs)
    print(f'Batch size {batch_size} end:   {time.strftime("%H:%M:%S")}')
    print(f'Epochs run: {len(history["loss"])}')
    print(f'model: {model_file}')
    print('---')
    del model
    del results

---
Batch size 1 start: 11:46:19


2022-01-28 11:46:20.192266: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-28 11:46:20.211086: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3398170000 Hz


Epoch 1/250


2022-01-28 11:46:21.171480: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-01-28 11:46:21.946074: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


171915/171915 [==============================] - 656s 4ms/step - loss: 0.2757 - mae: 0.2108 - root_mean_squared_error: 0.4888 - val_loss: 0.0068 - val_mae: 0.0450 - val_root_mean_squared_error: 0.0826
Epoch 2/250
171915/171915 [==============================] - 651s 4ms/step - loss: 0.0128 - mae: 0.0697 - root_mean_squared_error: 0.1131 - val_loss: 0.0049 - val_mae: 0.0346 - val_root_mean_squared_error: 0.0699
Epoch 3/250
171915/171915 [==============================] - 649s 4ms/step - loss: 0.0098 - mae: 0.0557 - root_mean_squared_error: 0.0988 - val_loss: 0.0043 - val_mae: 0.0310 - val_root_mean_squared_error: 0.0656
Epoch 4/250
 71369/171915 [===========>..................] - ETA: 5:33 - loss: 0.0091 - mae: 0.0509 - root_mean_squared_error: 0.0953